# pykabu-calendar Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obichan117/pykabu-calendar/blob/main/examples/quickstart.ipynb)

This notebook demonstrates how to use pykabu-calendar to get Japanese stock earnings calendar data.

## Installation

Install pykabu-calendar from PyPI:

In [ ]:
!pip install pykabu-calendar

## Basic Usage

Get the earnings calendar for a specific date:

In [ ]:
import pykabu_calendar as cal

# Get calendar for a specific date
# Uses lightweight sources by default (matsui, tradersweb)
df = cal.get_calendar("2026-02-10")

print(f"Found {len(df)} earnings announcements")
df.head(10)

## Output Schema

The calendar returns these columns:

| Column | Description |
|--------|-------------|
| `code` | Stock code (e.g., "7203") |
| `name` | Company name |
| `datetime` | Best estimate announcement datetime |
| `candidate_datetimes` | List of candidate datetimes (most likely first) |
| `sbi_datetime` | Datetime from SBI (if available) |
| `matsui_datetime` | Datetime from Matsui |
| `tradersweb_datetime` | Datetime from Tradersweb |
| `inferred_datetime` | Datetime inferred from historical patterns |
| `past_datetimes` | List of past earnings announcement datetimes |

In [ ]:
# View key columns
df[["code", "name", "datetime", "matsui_datetime", "tradersweb_datetime"]].head(10)

## Historical Inference

By default, pykabu-calendar infers announcement times based on past patterns using [pykabutan](https://pypi.org/project/pykabutan/).

You can disable this for faster results:

In [ ]:
# Disable historical inference for faster results
df_fast = cal.get_calendar("2026-02-10", infer_from_history=False)

print(f"Found {len(df_fast)} companies (without inference)")

In [ ]:
# With inference enabled (default), check inferred times and past patterns
df_inferred = cal.get_calendar("2026-02-10", infer_from_history=True)

# View companies with inferred datetimes
df_inferred[["code", "name", "datetime", "inferred_datetime", "past_datetimes"]].head(10)

## Data Sources

Different sources have different strengths:

| Source | Browser Required | Notes |
|--------|-----------------|-------|
| `matsui` | No | Lightweight, fast |
| `tradersweb` | No | Lightweight, fast |
| `sbi` | Yes (Playwright) | Most comprehensive |

In [ ]:
# Using only lightweight sources (default)
df_light = cal.get_calendar(
    "2026-02-10",
    sources=["matsui", "tradersweb"],
    infer_from_history=False
)
print(f"Lightweight sources: {len(df_light)} companies")

In [ ]:
# To include SBI (most comprehensive), install Playwright first:
# !pip install playwright
# !playwright install chromium

# Then use include_sbi=True:
# df_full = cal.get_calendar("2026-02-10", include_sbi=True)

## Individual Scrapers

You can also use individual scrapers directly:

In [ ]:
# Get data from Matsui only
df_matsui = cal.get_matsui("2026-02-10")
print(f"Matsui: {len(df_matsui)} companies")
df_matsui.head()

In [ ]:
# Get data from Tradersweb only
df_tradersweb = cal.get_tradersweb("2026-02-10")
print(f"Tradersweb: {len(df_tradersweb)} companies")
df_tradersweb.head()

## Inference Utilities

Use inference functions to predict announcement times based on historical patterns:

In [ ]:
# Get past earnings announcement times for a stock
past = cal.get_past_earnings("7203")  # Toyota
print(f"Past earnings announcements for Toyota (7203):")
for dt in past[:5]:
    print(f"  {dt}")

In [ ]:
# Infer datetime for a future announcement
inferred_dt, confidence, past_dts = cal.infer_datetime("7203", "2026-02-10")
print(f"Inferred datetime: {inferred_dt}")
print(f"Confidence: {confidence}")

## Export to CSV

Export the calendar to CSV for use in Excel or Google Sheets:

In [ ]:
# Export to CSV with proper encoding for Japanese characters
cal.export_to_csv(df, "earnings_2026-02-10.csv")

print("Exported to earnings_2026-02-10.csv")

In [ ]:
# Show all available columns
print("Available columns:")
print(df.columns.tolist())